# Unsupervised Learning Trading Strategy

* Download/Load SP500 stocks prices data.
* Calculate different features and indicators on each stock.
* Aggregate on monthly level and filter top 150 most liquid stocks.
* Calculate Monthly Returns for different time-horizons.
* Download Fama-French Factors and Calculate Rolling Factor Betas.
* For each month fit a K-Means Clustering Algorithm to group similar assets based on their features.
* For each month select assets based on the cluster and form a portfolio based on Efficient Frontier max sharpe ratio optimization.
* Visualize Portfolio returns and compare to SP500 returns.

* https://github.com/Luchkata/Algorithmic_Trading_Machine_Learning/blob/main/Algorithmic_Trading_Machine_Learning_Quant_Strategies.ipynb

In [6]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2023-09-27'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list, 
                 start=start_date, 
                 end=end_date).stack()

print(sp500)
df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()

df


[*********************100%%**********************]  503 of 503 completed

1 Failed download:
['VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 1443499200, endDate = 1695787200")


    Symbol              Security             GICS Sector  \
0      MMM                    3M             Industrials   
1      AOS           A. O. Smith             Industrials   
2      ABT                Abbott             Health Care   
3     ABBV                AbbVie             Health Care   
4      ACN             Accenture  Information Technology   
..     ...                   ...                     ...   
498    YUM           Yum! Brands  Consumer Discretionary   
499   ZBRA    Zebra Technologies  Information Technology   
500    ZBH         Zimmer Biomet             Health Care   
501   ZION  Zions Bancorporation              Financials   
502    ZTS                Zoetis             Health Care   

                      GICS Sub-Industry    Headquarters Location  Date added  \
0              Industrial Conglomerates    Saint Paul, Minnesota  1957-03-04   
1                     Building Products     Milwaukee, Wisconsin  2017-07-26   
2                 Health Care Equipment

Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.534702   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.684570   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.674236   52.790001   54.189999   51.880001   53.099998   
           ABT      33.644737   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 YUM     122.811577  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.992920  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZION     33.231339   33.990002   34.700001   33.840000   33.840000   
           ZTS     176.048248  176.869995  178.449997  176.270004  176.580002   

Price                   volume  
date       ticker               
2015-09-29 A         2252400.0  
           AAL       7478800.0  
           AAPL    293461600.0  
           ABBV     12842800.0  
           ABT      12287500.0  
...                        ...  
2023-09-26 YUM       1500600.0  
           ZBH       3610500.0  
           ZBRA       355400.0  
           ZION      1586100.0  
           ZTS       1463200.0  

[993179 rows x 6 columns]

In [7]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])

df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])

df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
             low=stock_data['low'],
             close=stock_data['close'],
             length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6
df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.534702   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.684570   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.674236   52.790001   54.189999   51.880001   53.099998   
           ABT      33.644737   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 YUM     122.811577  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.992920  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZION     33.231339   33.990002   34.700001   33.840000   33.840000   
           ZTS     176.048248  176.869995  178.449997  176.270004  176.580002   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-09-29 A         2252400.0         -0.000926        NaN       NaN   
           AAL       7478800.0         -0.000443        NaN       NaN   
           AAPL    293461600.0         -0.005572        NaN       NaN   
           ABBV     12842800.0         -0.051964        NaN       NaN   
           ABT      12287500.0         -0.008803        NaN       NaN   
...                        ...               ...        ...       ...   
2023-09-26 YUM       1500600.0          0.000002  36.057165  4.816569   
           ZBH       3610500.0          0.000161  31.893251  4.749948   
           ZBRA       355400.0          0.000133  29.494977  5.400991   
           ZION      1586100.0          0.000188  46.707767  3.517162   
           ZTS       1463200.0          0.000072  42.623479  5.158939   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2015-09-29 A            NaN       NaN       NaN       NaN      71.028763  
           AAL          NaN       NaN       NaN       NaN     279.420098  
           AAPL         NaN       NaN       NaN       NaN    7243.973499  
           ABBV         NaN       NaN       NaN       NaN     470.999882  
           ABT          NaN       NaN       NaN       NaN     413.409709  
...                     ...       ...       ...       ...            ...  
2023-09-26 YUM     4.846536  4.846536  0.142547 -1.363695     184.291052  
           ZBH     4.789615  4.789615 -0.381708 -0.881067     404.350437  
           ZBRA    5.539167  5.539167 -0.057389 -1.600791      79.595386  
           ZION    3.572582  3.572582 -0.161699 -0.164626      52.708226  
           ZTS     5.217753  5.217753  0.651515 -1.188278     257.593797  

[993179 rows x 14 columns]

# Aggregate to monthly level and filter top 150 most liquid stocks for each month.

In [8]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open', 'high', 'low', 'close']]



data = pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
           df.unstack()[last_cols].resample('M').last().stack('ticker')],
          axis=1).dropna()

data

dollar_volume   adj close  garman_klass_vol        rsi  \
date       ticker                                                           
2015-11-30 A          136.213753   39.086578         -0.001902  73.421499   
           AAL        287.915801   39.429935         -0.000966  40.718986   
           AAPL      4029.442284   26.890562         -0.003209  55.537326   
           ABBV       340.716450   40.770767         -0.056745  49.376868   
           ABT        212.708768   38.483475         -0.010572  56.962472   
...                          ...         ...               ...        ...   
2023-09-30 OTIS       154.983159   78.671936         -0.000020  33.116252   
           ABNB      1633.500725  132.279999          0.000213  44.494127   
           CEG        196.971016  108.026917          0.000171  55.245485   
           GEHC       212.350629   66.153511          0.000185  40.922336   
           KVUE       677.910279   20.218124          0.000032  35.706310   

                     bb_low    bb_mid   bb_high       atr      macd  
date       ticker                                                    
2015-11-30 A       3.547568  3.620020  3.620020 -1.033887  0.567157  
           AAL     3.672028  3.749832  3.749832  0.190822 -0.418771  
           AAPL    3.282983  3.326297  3.326297 -0.967900 -0.142789  
           ABBV    3.735768  3.784026  3.784026 -0.526809  0.145677  
           ABT     3.660875  3.682732  3.682732 -1.064842  0.335558  
...                     ...       ...       ...       ...       ...  
2023-09-30 OTIS    4.374104  4.419394  4.419394 -1.028320 -1.534536  
           ABNB    4.857047  4.940924  4.940924 -1.006939 -0.037854  
           CEG     4.653660  4.693834  4.693834 -0.436215  0.366876  
           GEHC    4.155782  4.213319  4.213319 -0.893478 -1.116463  
           KVUE    3.024299  3.099281  3.099281 -0.899746 -1.435621  

[46894 rows x 9 columns]

In [9]:
data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2016-10-31 AAL      39.134327         -0.000176  62.203542  3.604673   
           AAPL     26.248030         -0.002384  49.891115  3.295542   
           ABBV     40.620979         -0.044217  27.477778  3.762525   
           ABT      34.463535         -0.006968  38.008801  3.559438   
           ACN     103.970642         -0.004320  53.823653  4.641169   
...                       ...               ...        ...       ...   
2023-09-30 WMT      53.783062         -0.000011  54.722557  3.988367   
           XOM     114.292953          0.000026  59.440179  4.695106   
           MRNA     98.120003          0.000146  38.747314  4.582514   
           UBER     44.270000          0.000441  45.005268  3.806654   
           ABNB    132.279999          0.000213  44.494127  4.857047   

                     bb_mid   bb_high       atr      macd  
date       ticker                                          
2016-10-31 AAL     3.655493  3.655493  0.402199  1.131596  
           AAPL    3.324423  3.324423 -1.038688 -0.195978  
           ABBV    3.816700  3.816700 -0.893132 -0.760593  
           ABT     3.609919  3.609919 -1.035224 -0.650887  
           ACN     4.652807  4.652807 -0.996806 -0.135456  
...                     ...       ...       ...       ...  
2023-09-30 WMT     4.005838  4.005838 -0.196381  0.399459  
           XOM     4.735205  4.735205  0.601335  1.400623  
           MRNA    4.685332  4.685332 -0.529511 -0.376899  
           UBER    3.862227  3.862227 -0.746098 -0.133973  
           ABNB    4.940924  4.940924 -1.006939 -0.037854  

[12516 rows x 8 columns]

# calculate monthly returns for different time horizons as features

In [11]:
def calculate_returns(df):
    # g = df.xs('AAPL', level=1)
    outlier_cutoff = 0.005
    
    lags = [1,2,3,6,9,12]
    for lag in lags:
        df[f'return_{lag}m'] = (df['adj close']
                               .pct_change(lag)
                               .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                               .add(1)
                               .pow(1/lag)
                               .sub(1))
        return df


data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2016-12-31 AAL      45.118099         -0.000637  52.919078  3.813338   
           AAPL     26.912586         -0.002374  57.499151  3.273628   
           ABBV     45.602100         -0.039247  54.830570  3.800640   
           ABT      33.734570         -0.006171  45.324765  3.531072   
           ACN     104.766716         -0.005060  44.676141  4.637815   
...                       ...               ...        ...       ...   
2023-09-30 WMT      53.783062         -0.000011  54.722557  3.988367   
           XOM     114.292953          0.000026  59.440179  4.695106   
           MRNA     98.120003          0.000146  38.747314  4.582514   
           UBER     44.270000          0.000441  45.005268  3.806654   
           ABNB    132.279999          0.000213  44.494127  4.857047   

                     bb_mid   bb_high       atr      macd  return_1m  
date       ticker                                                     
2016-12-31 AAL     3.856552  3.856552  0.350459  0.992053   0.005383  
           AAPL    3.318435  3.318435 -1.076019 -0.114319   0.047955  
           ABBV    3.827259  3.827259 -1.249002 -0.030539   0.029934  
           ABT     3.550742  3.550742 -1.141478 -0.387895   0.008931  
           ACN     4.688794  4.688794 -0.862872 -0.362916  -0.019258  
...                     ...       ...       ...       ...        ...  
2023-09-30 WMT     4.005838  4.005838 -0.196381  0.399459  -0.000676  
           XOM     4.735205  4.735205  0.601335  1.400623   0.046947  
           MRNA    4.685332  4.685332 -0.529511 -0.376899  -0.132219  
           UBER    3.862227  3.862227 -0.746098 -0.133973  -0.062672  
           ABNB    4.940924  4.940924 -1.006939 -0.037854   0.005549  

[12145 rows x 9 columns]

# 5. Download Fama-French Factors and Calculate Rolling Factor Betas

In [ ]:
web.DataReader('F-F_Research_Data_5_Factors_2x3',
              )